# **Modelling and Tuning**

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV




In [3]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignore ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [4]:
import pickle

# Load train/val/test split data from notebook3
with open(r'Nata_Files\\train_test_split.pkl', 'rb') as f:
    notebook3_data = pickle.load(f)

# Core datasets
X = notebook3_data.get('X')
y = notebook3_data.get('y')

# Splits and processed feature sets
X_train = notebook3_data.get('X_train')
y_train = notebook3_data.get('y_train')
X_val = notebook3_data.get('X_val')
y_val = notebook3_data.get('y_val')
X_test = notebook3_data.get('X_test')
y_test = notebook3_data.get('y_test')
X_train_val = notebook3_data.get('X_train_val')
y_train_val = notebook3_data.get('y_train_val')

numeric_cols = notebook3_data.get('numeric_cols')
kf = notebook3_data.get('kf')
rkf = notebook3_data.get('rkf')
skf = notebook3_data.get('skf')

print("Train/Val/Test split data loaded successfully!")
if X is not None and y is not None:
    print(f"Full dataset X shape: {X.shape} | y shape: {y.shape}")
if X_train is not None:
    print(f"X_train shape: {X_train.shape}")
if X_val is not None:
    print(f"X_val shape: {X_val.shape}")
if X_test is not None:
    print(f"X_test shape: {X_test.shape}")
if kf is not None:
    try:
        print(f"kf splits: {kf.get_n_splits()}")
    except Exception:
        print("kf loaded (object), get_n_splits() unavailable for this object")
if rkf is not None:
    try:
        print(f"rkf splits: {rkf.get_n_splits()}")
    except Exception:
        print("rkf loaded (object), get_n_splits() unavailable for this object")

Train/Val/Test split data loaded successfully!
Full dataset X shape: (5196, 14) | y shape: (5196,)
X_train shape: (3117, 14)
X_val shape: (1039, 14)
X_test shape: (1040, 14)
kf splits: 10
rkf splits: 14


In [74]:
class Predictor:
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict_proba(self, X_val):
        return self.model.predict_proba(X_val)
    
    def predict(self, X_val):
        return self.model.predict(X_val)

## **Model Selection**

In [75]:
logr = Predictor(LogisticRegression())
logr.fit(X_train, y_train)
logr_proba = logr.predict_proba(X_val)[:,1]
logr_pred = logr.predict(X_val)
logr_proba_tr = logr.predict_proba(X_train)[:,1]
logr_pred_tr = logr.predict(X_train)

dtc = Predictor(DecisionTreeClassifier(max_depth= 5))
dtc.fit(X_train, y_train)
dtc_proba = dtc.predict_proba(X_val)[:,1]
dtc_pred = dtc.predict(X_val)
dtc_proba_tr = logr.predict_proba(X_train)[:,1]
dtc_pred_tr = logr.predict(X_train)


rf = Predictor(RandomForestClassifier())
rf.fit(X_train, y_train)
rf_proba = rf.predict_proba(X_val)[:,1]
rf_pred = rf.predict(X_val)
rf_proba_tr = logr.predict_proba(X_train)[:,1]
rf_pred_tr = logr.predict(X_train)

knn_clf = Predictor(KNeighborsClassifier())
knn_clf.fit(X_train, y_train)
knn_clf_proba = knn_clf.predict_proba(X_val)[:,1]
knn_clf_pred = knn_clf.predict(X_val)
knn_clf_proba_tr = logr.predict_proba(X_train)[:,1]
knn_clf_pred_tr = logr.predict(X_train)

lgb = LGBMClassifier(n_estimators=100, random_state=42)
lgb.fit(X_train, y_train)
lgb_proba = lgb.predict_proba(X_val)[:,1]
lgb_pred = lgb.predict(X_val)
lgb_proba_tr = logr.predict_proba(X_train)[:,1]
lgb_pred_tr = logr.predict(X_train)

[LightGBM] [Info] Number of positive: 1980, number of negative: 1137
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000499 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1624
[LightGBM] [Info] Number of data points in the train set: 3117, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.635226 -> initscore=0.554704
[LightGBM] [Info] Start training from score 0.554704


In [73]:
def get_metrics(y_val, y_proba, y_pred, model, dataset):
    return {
        "Model" : model,
        "Set" : dataset,
        "AUC": roc_auc_score(y_val, y_proba),
        "Accuracy": balanced_accuracy_score(y_val, y_pred),
    }

In [54]:
models_metrics = []

models_metrics.append(get_metrics(y_train, logr_proba_tr, logr_pred_tr, "Logistic Regression", "Train"))
models_metrics.append(get_metrics(y_train, dtc_proba_tr, dtc_pred_tr, "DTClassifier", "Train"))
models_metrics.append(get_metrics(y_train, rf_proba_tr, rf_pred_tr, "Random Forest", "Train"))
models_metrics.append(get_metrics(y_train, knn_clf_proba_tr, knn_clf_pred_tr, "KNClassifier", "Train"))
models_metrics.append(get_metrics(y_train, lgb_proba_tr, lgb_pred_tr, "LGBM", "Train"))

models_metrics.append(get_metrics(y_val, logr_proba, logr_pred, "Logistic Regression", "Validation"))
models_metrics.append(get_metrics(y_val, dtc_proba, dtc_pred, "DTClassifier", "Validation"))
models_metrics.append(get_metrics(y_val, rf_proba, rf_pred, "Random Forest", "Validation"))
models_metrics.append(get_metrics(y_val, knn_clf_proba, knn_clf_pred, "KNClassifier", "Validation"))
models_metrics.append(get_metrics(y_val, lgb_proba, lgb_pred, "LGBM", "Validation"))

In [55]:
df_models_metrics = pd.DataFrame(models_metrics)
df_models_metrics = df_models_metrics.pivot_table(
    index=["Model", "Set"],
    values=["AUC", "Accuracy"]
)

df_models_metrics

AUC  Accuracy
Model               Set                           
DTClassifier        Train       0.797320  0.702167
                    Validation  0.768210  0.703234
KNClassifier        Train       0.797320  0.702167
                    Validation  0.774752  0.690485
LGBM                Train       0.797320  0.702167
                    Validation  0.830563  0.733681
Logistic Regression Train       0.797320  0.702167
                    Validation  0.809067  0.716727
Random Forest       Train       0.797320  0.702167
                    Validation  0.838634  0.753496

#### So, we are going to create a one-level stacking that as LGBM and Random Forest as baseline models and Logistic Regression as metamodel

### **Hyperparameter Tuning**

### LightGBM

In [57]:
param_set = {
    'num_leaves': [15, 31, 63],
    'max_depth': [5, 10, -1], # -1 means no limit
    'learning_rate': [0.1, 0.03, 0.01],
    'n_estimators': [100, 200, 500],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'min_child_samples': [10, 20, 50, 100],
    'min_split_gain': [0.0, 0.1, 0.2],
}

lgb_clf = LGBMClassifier(random_state=42)
rs = RandomizedSearchCV(lgb_clf, param_set, n_iter= 20, cv = 3, scoring = 'balanced_accuracy')
rs.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 1320, number of negative: 758
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002094 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1598
[LightGBM] [Info] Number of data points in the train set: 2078, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.635226 -> initscore=0.554704
[LightGBM] [Info] Start training from score 0.554704
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

,estimator,LGBMClassifie...ndom_state=42)
,param_distributions,"{'colsample_bytree': [0.8, 1.0], 'learning_rate': [0.1, 0.03, ...], 'max_depth': [5, 10, ...], 'min_child_samples': [10, 20, ...], ...}"
,n_iter,20
,scoring,'balanced_accuracy'
,n_jobs,None
,refit,True
,cv,3
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


In [58]:
print("Best params:", rs.best_params_)

Best params: {'subsample': 0.8, 'num_leaves': 63, 'n_estimators': 200, 'min_split_gain': 0.0, 'min_child_samples': 100, 'max_depth': 10, 'learning_rate': 0.03, 'colsample_bytree': 0.8}


In [59]:
best_lgb = rs.best_estimator_

best_lgb_proba = best_lgb.predict_proba(X_val)[:,1]
best_lgb_pred = best_lgb.predict(X_val)

print(f"AUC: {roc_auc_score(y_val, best_lgb_proba)}, Accuracy: {accuracy_score(y_val, best_lgb_pred)}")

AUC: 0.8320460542096426, Accuracy: 0.7641963426371511


### Random Forest

In [64]:
param_range = { 
    'n_estimators': [100, 200, 400, 600],
    'max_depth': [None, 5, 10, 15],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2', 0.7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
    'class_weight': [None, 'balanced']
}

rfc = RandomForestClassifier(random_state=42)
clf = RandomizedSearchCV(rfc, param_range, n_iter= 20, cv = 3, scoring = 'balanced_accuracy', error_score= 'raise')
clf.fit(X_train,y_train)  

,estimator,RandomForestC...ndom_state=42)
,param_distributions,"{'bootstrap': [True, False], 'class_weight': [None, 'balanced'], 'criterion': ['gini', 'entropy'], 'max_depth': [None, 5, ...], ...}"
,n_iter,20
,scoring,'balanced_accuracy'
,n_jobs,None
,refit,True
,cv,3
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,'raise'


In [65]:
print("Best params:", clf.best_params_)

Best params: {'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 15, 'criterion': 'entropy', 'class_weight': 'balanced', 'bootstrap': False}


In [66]:
best_rfc = clf.best_estimator_

best_rfc_proba = best_rfc.predict_proba(X_val)[:,1]
best_rfc_pred = best_rfc.predict(X_val)

print(f"AUC: {roc_auc_score(y_val, best_rfc_proba)}, Accuracy: {accuracy_score(y_val, best_rfc_pred)}")

AUC: 0.8390861117774047, Accuracy: 0.7593840230991338


## Stacking

So now comes the part where we will combine our tuned models, and set them as baseline models to our Logisti Regression metamodel

In [67]:
estimators = [('rfc', best_rfc),
              ('lgb', best_lgb)]

In [69]:
stc = StackingClassifier(estimators= estimators, final_estimator=LogisticRegression()).fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 1980, number of negative: 1137
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000612 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1624
[LightGBM] [Info] Number of data points in the train set: 3117, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.635226 -> initscore=0.554704
[LightGBM] [Info] Start training from score 0.554704
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

In [71]:
stc_proba = stc.predict_proba(X_val)[:,1]
stc_pred = stc.predict(X_val)

print(f"AUC: {roc_auc_score(y_val, stc_proba)}, Accuracy: {accuracy_score(y_val, stc_pred)}")

AUC: 0.8401175341808588, Accuracy: 0.7718960538979788
